# Step 2: Assigning Speed to Traffic Data

In [81]:
import os, sys, time, importlib
import osmnx

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point

In [82]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
from GOSTnets.load_traffic2 import *

In [84]:
#read_nodes = pd.read_csv('./vavuniya_unclean_nodes.csv')

In [85]:
#read_nodes[:5]

In [86]:
# read graph
G = nx.read_gpickle('./sri_lanka_unclean2.pickle')

In [87]:
len(G.edges)

1930664

In [88]:
G[1650104033][6236632580]

AtlasView({0: {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695b50>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.009757789924575978}})

In [89]:
gn.example_edge(G, 5)

(1650104033, 6236632580, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695b50>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.009757789924575978})
(1650104033, 970058024, {'osm_id': 152177791, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695d10>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.0160020254611234})
(6236632580, 6236632579, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695dd0>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.005235256075996137})
(6236632580, 1650104033, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695b50>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.00975778992457

In [90]:
# speed dict based on existing speed limit tags
speed_dict_sri_lanka_mod1 = {
'primary': 60,
'primary_link': 50,
'motorway':100 ,
'motorway_link':35 ,
'secondary': 50,
'secondary_link': 45,
'tertiary':40,
'tertiary_link': 40,
'residential': 30,
'trunk': 60,
'trunk_link': 50,
'unclassified': 25,
'track': 25,
'service': 20
}

In [91]:
# modified the function below to calculate the correct time based on whether it has a Mapbox speed or not
# also it creates a new 'speed' attribute that is either based on the mapbox traffic speed, or the speed provided
# by the input dictionary if the mapbox traffic speed does not exist

In [92]:
def convert_network_to_time_w_traffic(G, distance_tag, graph_type = 'drive', road_col = 'highway', traffic_col = 'mean_speed', speed_dict = speed_dict_sri_lanka_mod1, walk_speed = 4.5, factor = 1, default = None, improvement_cost_per_km = None):
    """
    Function for adding a time value to edge dictionaries. Ensure any GeoDataFrames / graphs are in the same projection before using function, or pass a crs.

    DEFAULT SPEEDS:

               speed_dict = {
               'residential': 20,  # kmph
               'primary': 40, # kmph
               'primary_link':35,
               'motorway':50,
               'motorway_link': 45,
               'trunk': 40,
               'trunk_link':35,
               'secondary': 30,
               'secondary_link':25,
               'tertiary':30,
               'tertiary_link': 25,
               'unclassified':20
               }

    :param G: a graph containing one or more nodes
    :param distance_tag: the key in the dictionary for the field currently
               containing a distance in meters
    :param road_col: key for the road type in the edge data dictionary
    :param graph_type: set to either 'drive' or 'walk'. IF walk - will set time = walking time across all segment, using the supplied walk_speed. IF drive - will use a speed dictionary for each road type, or defaults as per the note below.
    :param speed_dict: speed dictionary to use. If not supplied, reverts to
               defaults
    :param walk_speed: specify a walkspeed in km/h
    :param factor: allows you to scale up / down distances if saved in a unit other than meters. Set to 1000 if length in km.
    :param default: if highway type not in the speed_dict, use this road class as an in-fill value for time.
    :returns: The original graph with a new data property for the edges called 'time'
    """

    if type(G) == nx.classes.multidigraph.MultiDiGraph or type(G) == nx.classes.digraph.DiGraph:
        pass
    else:
        raise ValueError('Expecting a graph or geodataframe for G!')

    import warnings

    try:
        # checks the first edge to see if the 'time' attribute already exists
        if list(G.edges(data = True))[0][2]['time']:
          warnings.warn('Aree you sure you want to convert length to time? This graph already has a time attribute')
    except:
        pass

    G_adj = G.copy()

    for u, v, data in G_adj.edges(data=True):

        # the default lenth in the graph should be in km
        orig_len = data[distance_tag]

        # Note that this is a MultiDiGraph so there could
        # be multiple indices here, I naively assume this is not
        # the case
        data['length'] = orig_len * factor
        

        # get appropriate speed limit
        if graph_type == 'walk':
            speed = walk_speed

        elif graph_type == 'drive':

            if speed_dict == None:
                speed_dict = {
                'residential': 20,  # kmph
                'primary': 40, # kmph
                'primary_link':35,
                'motorway':50,
                'motorway_link': 45,
                'trunk': 40,
                'trunk_link':35,
                'secondary': 30,
                'secondary_link':25,
                'tertiary':30,
                'tertiary_link': 25,
                'unclassified':20
                }

            highwayclass = data[road_col]
            
            trafficclass = data[traffic_col]

            if trafficclass > 0:
                speed = data[traffic_col]
                data['speed'] = speed
            else:
                if type(highwayclass) == list:
                    highwayclass = highwayclass[0]

                if highwayclass in speed_dict.keys():
                    speed = speed_dict[highwayclass]
                else:
                    if default == None:
                        speed = 20
                    else:
                        speed = speed_dict[default]
                data['speed'] = speed
            
            #calculate road improvement costs for secondary or lower highways
            if highwayclass in ['seconday', 'secondary_link', 'tertiary', 'tertiary_link', 'unclassified', 'residential']:
                data['imp_cost'] = orig_len * improvement_cost_per_km

        else:
            raise ValueError('Expecting either a graph_type of "walk" or "drive"!')

        # perform conversion
        hours = orig_len / speed
        in_seconds = hours * 60 * 60
        data['time'] = in_seconds

        # And state the mode, too
        data['mode'] = graph_type

    return G_adj

In [93]:
G_time = convert_network_to_time_w_traffic(G, distance_tag = 'length', road_col = 'infra_type', factor = 1, improvement_cost_per_km = 186800)

In [94]:
gn.example_edge(G_time, 15)

(1650104033, 6236632580, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695b50>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.009757789924575978, 'speed': 26.793650793650794, 'time': 1.3110585040840237, 'mode': 'drive'})
(1650104033, 970058024, {'osm_id': 152177791, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695d10>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.0160020254611234, 'speed': 26.793650793650794, 'time': 2.150035174515869, 'mode': 'drive'})
(6236632580, 6236632579, {'osm_id': 4860427, 'geometry': <shapely.geometry.linestring.LineString object at 0x7f6a4c695dd0>, 'infra_type': 'trunk', 'min_speed': 23.0, 'max_speed': 37.0, 'mean_speed': 26.793650793650794, 'length': 0.005235256075996137, 'speed': 26.793650793650794, 'time': 0.7034099988364477, 'mode': 'drive'})
(6236632580, 1650104033,

In [95]:
#save graph again
#gn.save(G_time,'sri_lanka_unclean2_w_time','./', pickle = True, edges = True, nodes = True)

## Export edges as shapefile to visualize

In [96]:
#edge_gdf_w_traffic = gn.edge_gdf_from_graph(G_time)

In [97]:
#edge_gdf_w_traffic

In [98]:
#edge_gdf_w_traffic.to_file(driver = 'ESRI Shapefile', filename = './sri_lanka_hwy_w_traffic_and_time2.shp')

### now take the biggest sub-graph and compare

In [99]:
# before
# let's print info on our clean version
print(nx.info(G_time))

Name: 
Type: MultiDiGraph
Number of nodes: 1052094
Number of edges: 1930664
Average in degree:   1.8351
Average out degree:   1.8351


In [100]:
# Identify only the largest graph

# compatible with NetworkX 2.4
list_of_subgraphs = list(G_time.subgraph(c).copy() for c in nx.strongly_connected_components(G_time))
max_graph = None
max_edges = 0
for i in list_of_subgraphs:
    if i.number_of_edges() > max_edges:
        max_edges = i.number_of_edges()
        max_graph = i

# set your graph equal to the largest sub-graph
G_largest = max_graph

In [101]:
# print info about the largest sub-graph
print(nx.info(G_largest))

Name: 
Type: MultiDiGraph
Number of nodes: 820346
Number of edges: 1675316
Average in degree:   2.0422
Average out degree:   2.0422


In [102]:
# re-save
#edge_gdf_w_traffic_largest = gn.edge_gdf_from_graph(G_largest)

In [103]:
#edge_gdf_w_traffic_largest.to_file(driver = 'ESRI Shapefile', filename = './sri_lanka_hwy_w_traffic_and_time2_largest.shp')

In [104]:
#save graph again
gn.save(G_largest,'sri_lanka_unclean2_w_time_largest_20200616','./', pickle = True, edges = True, nodes = True)